# Training Music Classification Models

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from datetime import datetime
from pathlib import Path
import random

def set_seed(seed=42):
    """Set seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

## Training Configuration

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Training hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.0001  # Lowered from 0.001
NUM_EPOCHS = 50
EARLY_STOPPING_PATIENCE = 10

Using device: cuda


In [ ]:
# Setup run directory
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = Path(f"../runs/{run_id}")
run_dir.mkdir(parents=True, exist_ok=True)
print(f"Run directory created at: {run_dir}")

# Create changes.md
changes_file = run_dir / "changes.md"
with open(changes_file, "w") as f:
    f.write(f"# Run {run_id}\n\n")
    f.write("## Configuration\n")
    f.write(f"- Batch Size: {BATCH_SIZE}\n")
    f.write(f"- Learning Rate: {LEARNING_RATE}\n")
    f.write(f"- Epochs: {NUM_EPOCHS}\n")
    f.write(f"- Device: {device}\n")
    f.write(f"- Data Strategy: Chunking (3s chunks, 50% overlap)\n")
    f.write(f"- Augmentation: Noise=0.01, Shift=0.3\n")
    f.write(f"- Optimization: In-memory caching + Mixed Precision (AMP)\n")
    f.write(f"- Stability: Seed=42, Weight Decay=1e-4 (Standard), Gradient Clipping=1.0\n")
    f.write(f"- Data Split: Stratified (Balanced Validation Set)\n\n")
    f.write("## Changes\n")
    f.write("- Added more dropout layers to the ImprovedCNN model to reduce overfitting and removed layer 4 in the model.\n\n")
    f.write("## Results\n")

Run directory created at: ..\runs\20251127_172211


## Training Function (Single-label Classification)

In [4]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Use mixed precision training
    scaler = torch.amp.GradScaler()
    
    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass with mixed precision
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        scaler.scale(loss).backward()
        
        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({'loss': loss.item(), 'acc': 100 * correct / total})
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

In [5]:
def validate_epoch(model, val_loader, criterion, device):
    """Validate for one epoch."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': loss.item(), 'acc': 100 * correct / total})
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc, all_preds, all_labels

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs, learning_rate, device, 
                save_path='../models/best_model.pth', changes_file=None):
    """Complete training loop with early stopping."""
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    # Reduced weight decay back to 1e-4 as we have more data now
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, 
                                                       patience=5)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 50)
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss, val_acc, _, _ = validate_epoch(model, val_loader, criterion, device)
        
        # Update scheduler
        scheduler.step(val_loss)
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), save_path)
            print(f"✓ Model saved to {save_path}")
        else:
            patience_counter += 1
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    if changes_file:
        with open(changes_file, "a") as f:
            f.write(f"- Final Train Loss: {history['train_loss'][-1]:.4f}\n")
            f.write(f"- Final Val Loss: {history['val_loss'][-1]:.4f}\n")
            f.write(f"- Final Train Acc: {history['train_acc'][-1]:.2f}%\n")
            f.write(f"- Final Val Acc: {history['val_acc'][-1]:.2f}%\n")

    return history

## Training Function (Multi-label Classification)

In [7]:
def train_multilabel(model, train_loader, val_loader, num_epochs, learning_rate, device,
                     save_path='../models/best_model_multilabel.pth'):
    """Training loop for multi-label classification."""
    model = model.to(device)
    
    # Loss and optimizer (BCE for multi-label)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,
                                                       patience=5)
    
    history = {
        'train_loss': [],
        'val_loss': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 50)
        
        # Training
        model.train()
        train_loss = 0.0
        train_batches = 0
        
        pbar = tqdm(train_loader, desc='Training')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_batches += 1
            pbar.set_postfix({'loss': loss.item()})
        
        train_loss /= train_batches
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_batches = 0
        
        with torch.no_grad():
            pbar = tqdm(val_loader, desc='Validation')
            for inputs, labels in pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_batches += 1
                pbar.set_postfix({'loss': loss.item()})
        
        val_loss /= val_batches
        
        scheduler.step(val_loss)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {val_loss:.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"✓ Model saved to {save_path}")
        else:
            patience_counter += 1
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    return history

## Plot Training History

In [8]:
def plot_training_history(history, multi_label=False, save_path=None):
    """Plot training history."""
    fig, axes = plt.subplots(1, 2 if not multi_label else 1, figsize=(15, 5))
    
    if not multi_label:
        # Loss plot
        axes[0].plot(history['train_loss'], label='Train Loss')
        axes[0].plot(history['val_loss'], label='Val Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Training and Validation Loss')
        axes[0].legend()
        axes[0].grid(True)
        
        # Accuracy plot
        axes[1].plot(history['train_acc'], label='Train Accuracy')
        axes[1].plot(history['val_acc'], label='Val Accuracy')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy (%)')
        axes[1].set_title('Training and Validation Accuracy')
        axes[1].legend()
        axes[1].grid(True)
    else:
        # Loss plot only for multi-label
        axes.plot(history['train_loss'], label='Train Loss')
        axes.plot(history['val_loss'], label='Val Loss')
        axes.set_xlabel('Epoch')
        axes.set_ylabel('Loss')
        axes.set_title('Training and Validation Loss')
        axes.legend()
        axes.grid(True)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

### For Single-label Classification (GTZAN, FMA)

In [ ]:
# Train SimpleCNN on GTZAN

# Ensure repository root is on sys.path
import os
import sys
from pathlib import Path
repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Import model (prefer module; fallback to notebook)
try:
    from model_cnn import ImprovedCNN
except ModuleNotFoundError:
    print("Model module not found; loading from notebook via %run ...")
    %run "./04_model_cnn.ipynb"

# Import dataset from stable utils module (Windows-safe)
try:
    from utils.datasets_gtzan import GTZANDataset, create_dataloaders, GENRES, AudioAugmentation
except ModuleNotFoundError:
    print("Dataset module not found; loading from notebook via %run ...")
    %run "./01_data_loading_gtzan.ipynb"

# Create dataset with in-memory caching
gtzan_root = repo_root / "data" / "gtzan"
dataset = GTZANDataset(str(gtzan_root), cache_to_memory=True)
print(f"GTZAN files: {len(dataset)}")

# Define augmentation
train_transform = AudioAugmentation(noise_level=0.01, shift_max=0.3)

# Create loaders with Stratified Split AND Chunking
# NOTE: With cache_to_memory=True, we must use num_workers=0 on Windows to avoid 
# pickling the entire cached dataset to worker processes, which causes hangs/OOM.
train_loader, val_loader = create_dataloaders(
    dataset, 
    batch_size=BATCH_SIZE, 
    num_workers=0,
    train_transform=train_transform,
    chunk_length_sec=3.0 # Enable chunking
)

# Create model
model = ImprovedCNN(n_classes=10)

# Train
history = train_model(
    model, train_loader, val_loader,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    device=device,
    save_path=str(run_dir / 'gtzan_cnn.pth'),
    changes_file=changes_file
)

# Plot results
plot_training_history(history, save_path=str(run_dir / 'training_history.png'))

Model module not found; loading from notebook via %run ...
SimpleCNN:
SimpleCNN(
  (mel_spec): MelSpectrogram(
    (spectrogram): Spectrogram()
    (mel_scale): MelScale()
  )
  (amplitude_to_db): AmplitudeToDB()
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, ke

C:\Users\mar20\AppData\Local\Temp\ipykernel_2828\660122760.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Caching complete.
GTZAN files: 999
Created stratified split: 799 train songs, 200 val songs
Applying chunking: 3.0s chunks with 50% overlap
Chunked dataset sizes: 15181 train chunks, 3800 val chunks

Epoch 1/50
--------------------------------------------------


Validation: 100%|██████████| 119/119 [00:02<00:00, 41.83it/s, loss=0.966, acc=45.6]


Train Loss: 1.7765, Train Acc: 41.95%
Val Loss: 2.0315, Val Acc: 45.63%
✓ Model saved to ..\runs\20251127_172211\gtzan_cnn.pth

Epoch 2/50
--------------------------------------------------


...

Train Loss: 0.7119, Train Acc: 94.11%
Val Loss: 1.4534, Val Acc: 67.03%

Epoch 28/50
--------------------------------------------------


Validation: 100%|██████████| 119/119 [00:02<00:00, 55.93it/s, loss=1.13, acc=70.7] 


Train Loss: 0.7145, Train Acc: 94.25%
Val Loss: 1.3126, Val Acc: 70.68%

Epoch 29/50
--------------------------------------------------


Validation: 100%|██████████| 119/119 [00:02<00:00, 55.38it/s, loss=1.3, acc=67.5]  


Train Loss: 0.7059, Train Acc: 94.64%
Val Loss: 1.4643, Val Acc: 67.53%

Early stopping triggered after 29 epochs


### For Multi-label Classification (MTAT)

In [ ]:
# Example: Train DeepCNN on MTAT
# Uncomment and adapt to your dataset

# from notebooks.model_cnn import DeepCNN
# from notebooks.data_loading_mtat import MTATDataset, create_dataloaders

# # Create dataset
# dataset = MTATDataset(MTAT_AUDIO_PATH, MTAT_ANNOTATIONS_PATH, top_tags=50)
# train_loader, val_loader = create_dataloaders(dataset, batch_size=BATCH_SIZE)

# # Create model
# model = DeepCNN(n_classes=50)

# # Train
# history = train_multilabel(
#     model, train_loader, val_loader,
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     device=device,
#     save_path='../models/mtat_cnn.pth'
# )

# # Plot results
# plot_training_history(history, multi_label=True)

## Evaluation Metrics

In [ ]:
def evaluate_model(model, test_loader, device, genre_names=None, changes_file=None):
    """Evaluate model and print detailed metrics."""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print(f"\nTest Metrics:")
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    if changes_file:
        with open(changes_file, "a") as f:
            f.write(f"- Test Accuracy: {accuracy*100:.2f}%\n")
            f.write(f"- Test Precision: {precision:.4f}\n")
            f.write(f"- Test Recall: {recall:.4f}\n")
            f.write(f"- Test F1-Score: {f1:.4f}\n")
    
    return all_preds, all_labels


evaluate_model(
    model, val_loader, device, genre_names=GENRES, changes_file=changes_file
)